<img src="Images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

# Assessment: Accelerate and Optimize Maxwell's Equations Simulator

The [Maxwell's Equations](https://en.wikipedia.org/wiki/Maxwell%27s_equations) simulator predicts how electromagnetic waves propagate. 
For this assessment, you'll begin with a simple, though working, 2D Maxwell's Equations simulator. In its current CPU-only form, this application takes about 15 seconds to run on 4096^2 cells, and 4 minutes to run on 65536^2 cells. 
Your task is to GPU-accelerate the program, retaining the correctness of the simulation.

### Table of Contents
[The Problem](#The-Problem)<br>
[Scoring](#Scoring)<br>
[Step 1: Port CPU Simulator to GPU](#Step-1:-Port-CPU-Simulator-to-GPU)<br>
[Step 2: Accelerate the Simulator with Fancy Iterators](#Step-2:-Accelerate-the-Simulator-with-Fancy-Iterators)<br>
[Step 3: Coarse Grid with CUDA Kernel](#Step-3:-Coarse-Grid-with-CUDA-Kernel)<br>
[Step 4: Submit Your Assessment](#Step-4:-Submit-Your-Assessment)

---
# The Problem

The provided code simulates the propagation of electromagnetic waves in a 2D grid using Maxwell's equations. It performs iterative updates of the electric and magnetic fields, accounting for spatial and temporal changes.  The use of CPU-based computation limits the performance as it is scaled up.  Your job is to accelerate the computation using techniques learned in this class. 

---
# Scoring
You will be assessed on your ability to accelerate the simulator in three steps, from basic parallelization to employing fancy iterators, and finally using a course grid kernel.   This coding assessment is worth 100 points, divided as follows:

### Rubric

| Step                                              | FIXMEs?  | Points |
|---------------------------------------------------|----------|--------|
| 1. Port CPU Simulator to GPU                      |  2       | 50     |
| 2. Accelerate the Simulator with Fancy Iterators |  3       | 25     |
| 3. Use Cooperative Algorithms to Coarse the Grid  |  3       | 25     |

Although you are very capable at this point of building the project without any help at all, some scaffolding is provided.
Sources for each of the steps have a few FIXMEs to help you focus on the most important changes.

Once you are confident that you've built a correct and optimized application, follow the instructions for submission (Step 4) at the end of the notebook.

### Resources and Hints

**Step 1**
  * Refer to [1.2 Execution Spaces](../01.02-Execution-Spaces/01.02.01-Execution-Spaces.ipynb) for guidance on how to port CPU algorithms to GPU.
  * Refer to [1.6 Memory Spaces](../01.06-Memory-Spaces/01.06.01-Memory-Spaces.ipynb) for guidance on how to leverage explicit memory spaces.
  * For your initial refactors, aim to keep the application's logic, especially the lambda functions, mostly unchanged. Focus on accelerating it in the simplest way possible.
  * After your changes, all `FIXME(Step 1)` comments should be addressed.

**Step 2**
  * Refer to [1.3 Extending Algorithms](../01.03-Extending-Algorithms/01.03.01-Extending-Algorithms.ipynb) for guidance on how to avoid materialization of temporary values in memory.
  * After your changes, all `FIXME(Step 2)` comments should be addressed.

**Step 3**
  * Refer to [3.6 Cooperative Algorithms](../03.06-Cooperative-Algorithms/03.06.01-Cooperative.ipynb) for guidance on how to avoid materialization of temporary values in memory.

**Have Fun!**


---

# Step 1: Port CPU Simulator to GPU

Begin by importing the assessment evaluator code and a fresh copy of the the simple, CPU-only version of the simulator into your workspace.

In [ ]:
# Import the evaluator Python code
import Sources.dli

Let's start by checking the throughput and visualizing the results of the Maxwell's Equations simulator. 
Run the cell below to check CPU performance and visualize the results. 
The entire simulator has already been ported to GPU, so the initial state for the simulation is provided as device containers. 

However, `simulate()` is still lacking GPU acceleration. 
It copies data to *host*, performs the simulation, and copies data back to *device*.
Your task is to get rid of all copies to *host* and run the simulation in the provided *device* containers instead.

```c++
// Do not change the signature of this function
void simulate(int cells_along_dimension, float dx, float dy, float dt,
              thrust::device_vector<float> &d_hx,
              thrust::device_vector<float> &d_hy,
              thrust::device_vector<float> &d_dz,
              thrust::device_vector<float> &d_ez) 
{
  // Remove host containers and compute in the incoming device containers
  std::vector<float> hx = copy_to_host(d_hx);
  std::vector<float> hy = copy_to_host(d_hy);
  std::vector<float> dz = copy_to_host(d_dz);
  std::vector<float> ez = copy_to_host(d_ez);

  int cells = cells_along_dimension * cells_along_dimension;
  std::vector<float> buffer(cells);

  // Materialize cell indices (`i`) in memory
  std::vector<int> cell_ids(cells);
  for (int i = 0; i < cells; i++) {
    cell_ids[i] = i;
  }

  for (int step = 0; step < dli::steps; step++) {
    update_hx(cells_along_dimension, dx, dy, dt, hx, ez, buffer);
    update_hy(cells_along_dimension, dx, dy, dt, hy, ez, buffer);
    update_dz(cells_along_dimension, dx, dy, dt, hx, hy, dz, cell_ids);
    update_ez(ez, dz);
  }

  // Copy back to device
  d_hx = hx;
  d_hy = hy;
  d_dz = dz;
  d_ez = ez;
}
```

Nevertheless, as written, the simulator works correctly.
Below you can see how electromagnetic waves propagate in a 2D grid.

In [ ]:
Sources.dli.run_step_1()

To GPU-accelerate `simulate()`, please edit the code below to port all algorithms to GPU using Thrust.
Your changes shouldn't affect visualization, but you are expected to improve the throughput that the simulator reports. 

If you want to start again, just copy the backup source [Backup/maxwell.cu](Backup/maxwell.cu).

<details>
<summary>Original code in case you need it</summary>

```c++
%%writefile Sources/maxwell.cu
#include "dli.h"

// FIXME(Step 1):
// accept device containers instead of `std::vector<float>`
void update_hx(int n, float dx, float dy, float dt, std::vector<float> &hx,
               std::vector<float> &ez, std::vector<float> &buffer) {
  // FIXME(Step 2):
  // Use zip and transform iterators to avoid materializing `ez[i + n] - ez[i]`
  // FIXME(Step 1):
  // compute transformation on GPU
  std::transform(ez.begin() + n, ez.end(), ez.begin(), buffer.begin(),
                 [](float x, float y) { return x - y; });

  // FIXME(Step 1):
  // compute transformation on GPU
  std::transform(hx.begin(), hx.end() - n, buffer.begin(), hx.begin(),
                 [dt, dx, dy](float h, float cex) {
                   return h - dli::C0 * dt / 1.3f * cex / dy;
                 });
}

// FIXME(Step 1):
// accept device containers instead of `std::vector<float>`
void update_hy(int n, float dx, float dy, float dt, std::vector<float> &hy,
               std::vector<float> &ez, std::vector<float> &buffer) {
  // FIXME(Step 2):
  // Use zip and transform iterators to avoid materializing `ez[i] - ez[i + 1]`
  // FIXME(Step 1):
  // compute transformation on GPU
  std::transform(ez.begin(), ez.end() - 1, ez.begin() + 1, buffer.begin(),
                 [](float x, float y) { return x - y; });

  // FIXME(Step 1):
  // compute transformation on GPU
  std::transform(hy.begin(), hy.end() - 1, buffer.begin(), hy.begin(),
                 [dt, dx, dy](float h, float cey) {
                   return h - dli::C0 * dt / 1.3f * cey / dx;
                 });
}

// FIXME(Step 1):
// accept device containers instead of `std::vector<float>`
void update_dz(int n, float dx, float dy, float dt, std::vector<float> &hx_vec,
               std::vector<float> &hy_vec, std::vector<float> &dz_vec,
               std::vector<int> &cell_ids) {
  auto hx = hx_vec.begin();
  auto hy = hy_vec.begin();
  auto dz = dz_vec.begin();

  // FIXME(Step 1):
  // compute for each on GPU
  std::for_each(cell_ids.begin(), cell_ids.end(),
                [n, dx, dy, dt, hx, hy, dz](int cell_id) {
                  if (cell_id > n) {
                    float hx_diff = hx[cell_id - n] - hx[cell_id];
                    float hy_diff = hy[cell_id] - hy[cell_id - 1];
                    dz[cell_id] += dli::C0 * dt * (hx_diff / dx + hy_diff / dy);
                  }
                });
}

// FIXME(Step 1):
// accept device containers instead of `std::vector<float>`
void update_ez(std::vector<float> &ez, std::vector<float> &dz) {
  // FIXME(Step 1):
  // compute transformation on GPU
  std::transform(dz.begin(), dz.end(), ez.begin(),
                 [](float d) { return d / 1.3f; });
}

// FIXME(Step 1):
// remove this function
std::vector<float> copy_to_host(const thrust::device_vector<float> &d_vec) {
  std::vector<float> vec(d_vec.size());
  thrust::copy(d_vec.begin(), d_vec.end(), vec.begin());
  return vec;
}

// Do not change the signature of this function
void simulate(int cells_along_dimension, float dx, float dy, float dt,
              thrust::device_vector<float> &d_hx,
              thrust::device_vector<float> &d_hy,
              thrust::device_vector<float> &d_dz,
              thrust::device_vector<float> &d_ez) {
  // FIXME(Step 1):
  // remove host containers and compute in the incoming device containers
  std::vector<float> hx = copy_to_host(d_hx);
  std::vector<float> hy = copy_to_host(d_hy);
  std::vector<float> dz = copy_to_host(d_dz);
  std::vector<float> ez = copy_to_host(d_ez);

  // FIXME(Step 2):
  // Remove `cell_ids` vector and use counting iterator instead
  int cells = cells_along_dimension * cells_along_dimension;
  std::vector<int> cell_ids(cells);
  for (int i = 0; i < cells; i++) {
    cell_ids[i] = i;
  }

  // FIXME(Step 2):
  // Remove `buffer` vector and use fancy iterators instead
  std::vector<float> buffer(cells);

  for (int step = 0; step < dli::steps; step++) {
    update_hx(cells_along_dimension, dx, dy, dt, hx, ez, buffer);
    update_hy(cells_along_dimension, dx, dy, dt, hy, ez, buffer);
    update_dz(cells_along_dimension, dx, dy, dt, hx, hy, dz, cell_ids);
    update_ez(ez, dz);
  }

  // FIXME(Step 1):
  // remove copy to host containers, compute in the incoming device containers
  d_hx = hx;
  d_hy = hy;
  d_dz = dz;
  d_ez = ez;
}
```

</details>

In [ ]:
%%writefile Sources/maxwell.cu
#include "dli.h"

// FIXME(Step 1):
// accept device containers instead of `std::vector<float>`
void update_hx(int n, float dx, float dy, float dt, std::vector<float> &hx,
               std::vector<float> &ez, std::vector<float> &buffer) {
  // FIXME(Step 2):
  // Use zip and transform iterators to avoid materializing `ez[i + n] - ez[i]`
  // FIXME(Step 1):
  // compute transformation on GPU
  std::transform(ez.begin() + n, ez.end(), ez.begin(), buffer.begin(),
                 [](float x, float y) { return x - y; });

  // FIXME(Step 1):
  // compute transformation on GPU
  std::transform(hx.begin(), hx.end() - n, buffer.begin(), hx.begin(),
                 [dt, dx, dy](float h, float cex) {
                   return h - dli::C0 * dt / 1.3f * cex / dy;
                 });
}

// FIXME(Step 1):
// accept device containers instead of `std::vector<float>`
void update_hy(int n, float dx, float dy, float dt, std::vector<float> &hy,
               std::vector<float> &ez, std::vector<float> &buffer) {
  // FIXME(Step 2):
  // Use zip and transform iterators to avoid materializing `ez[i] - ez[i + 1]`
  // FIXME(Step 1):
  // compute transformation on GPU
  std::transform(ez.begin(), ez.end() - 1, ez.begin() + 1, buffer.begin(),
                 [](float x, float y) { return x - y; });

  // FIXME(Step 1):
  // compute transformation on GPU
  std::transform(hy.begin(), hy.end() - 1, buffer.begin(), hy.begin(),
                 [dt, dx, dy](float h, float cey) {
                   return h - dli::C0 * dt / 1.3f * cey / dx;
                 });
}

// FIXME(Step 1):
// accept device containers instead of `std::vector<float>`
void update_dz(int n, float dx, float dy, float dt, std::vector<float> &hx_vec,
               std::vector<float> &hy_vec, std::vector<float> &dz_vec,
               std::vector<int> &cell_ids) {
  auto hx = hx_vec.begin();
  auto hy = hy_vec.begin();
  auto dz = dz_vec.begin();

  // FIXME(Step 1):
  // compute for each on GPU
  std::for_each(cell_ids.begin(), cell_ids.end(),
                [n, dx, dy, dt, hx, hy, dz](int cell_id) {
                  if (cell_id > n) {
                    float hx_diff = hx[cell_id - n] - hx[cell_id];
                    float hy_diff = hy[cell_id] - hy[cell_id - 1];
                    dz[cell_id] += dli::C0 * dt * (hx_diff / dx + hy_diff / dy);
                  }
                });
}

// FIXME(Step 1):
// accept device containers instead of `std::vector<float>`
void update_ez(std::vector<float> &ez, std::vector<float> &dz) {
  // FIXME(Step 1):
  // compute transformation on GPU
  std::transform(dz.begin(), dz.end(), ez.begin(),
                 [](float d) { return d / 1.3f; });
}

// FIXME(Step 1):
// remove this function
std::vector<float> copy_to_host(const thrust::device_vector<float> &d_vec) {
  std::vector<float> vec(d_vec.size());
  thrust::copy(d_vec.begin(), d_vec.end(), vec.begin());
  return vec;
}

// Do not change the signature of this function
void simulate(int cells_along_dimension, float dx, float dy, float dt,
              thrust::device_vector<float> &d_hx,
              thrust::device_vector<float> &d_hy,
              thrust::device_vector<float> &d_dz,
              thrust::device_vector<float> &d_ez) {
  // FIXME(Step 1):
  // remove host containers and compute in the incoming device containers
  std::vector<float> hx = copy_to_host(d_hx);
  std::vector<float> hy = copy_to_host(d_hy);
  std::vector<float> dz = copy_to_host(d_dz);
  std::vector<float> ez = copy_to_host(d_ez);

  // FIXME(Step 2):
  // Remove `cell_ids` vector and use counting iterator instead
  int cells = cells_along_dimension * cells_along_dimension;
  std::vector<int> cell_ids(cells);
  for (int i = 0; i < cells; i++) {
    cell_ids[i] = i;
  }

  // FIXME(Step 2):
  // Remove `buffer` vector and use fancy iterators instead
  std::vector<float> buffer(cells);

  for (int step = 0; step < dli::steps; step++) {
    update_hx(cells_along_dimension, dx, dy, dt, hx, ez, buffer);
    update_hy(cells_along_dimension, dx, dy, dt, hy, ez, buffer);
    update_dz(cells_along_dimension, dx, dy, dt, hx, hy, dz, cell_ids);
    update_ez(ez, dz);
  }

  // FIXME(Step 1):
  // remove copy to host containers, compute in the incoming device containers
  d_hx = hx;
  d_hy = hy;
  d_dz = dz;
  d_ez = ez;
}

Once you address all `FIXME(Step 1)` comments, run the code cell below to see how your changes affect performance of the application.  
The result will automatically be recorded for the autograder.

In [ ]:
# Do not change this cell; results are recorded for the autograder
Sources.dli.passes_step_1_with_n_of(1024)

### Check Your Work
Was the expected throughput achieved? 
Was the result still accurate? 
If not, try again until your code is fast enough and still correct. 
If your code passed, move on to the Step 2. 


---

# Step 2: Accelerate the Simulator with Fancy Iterators

As written, the simulator is functional and GPU-accelerated, but it can be improved further.
For instance, consider code below:

```c++
// Materialize `ez[i] - ez[i + 1]` in buffer
std::transform(ez.begin(), ez.end() - 1, ez.begin() + 1, buffer.begin(),
                [](float x, float y) { return x - y; });

std::transform(hy.begin(), hy.end() - 1, buffer.begin(), hy.begin(),
                [dt, dx, dy](float h, float cey) {
                  return h - dli::C0 * dt / 1.3f * cey / dx;
                });
```

We don't actually need data written to `buffer` to be materialized in memory.
These are just temporary values that are used to compute `hy`.
Besides that, there's a better way to provide cell indices to parallel algorithms than materializing them in memory:

```cpp
// Materialize cell indices (`i`) in memory
std::vector<int> cell_ids(cells);
for (int i = 0; i < cells; i++) {
  cell_ids[i] = i;
}
```

For this step, you'll have to remove any memory allocations from the code.
Please copy your current solution from previous source or [Sources/maxwell.cu](Sources/maxwell.cu) into the following cell.
Then, use counting, zip, and transform iterators instead of materialized vectors.

In [ ]:
%%writefile Sources/maxwell.cu
#include "dli.h"

// 1. Copy content of the previous cell to this one 
// 2. Use counting, zip, and transform iterators instead of materialized vectors
// 3. Run this cell to save the changes

Run the code cell below to see how your changes affect performance of the application.  

In [ ]:
Sources.dli.run_step_2()

Once you address all `FIXME(Step 2)` comments, run the code cell below to see how your changes affect performance of the application.
The result will automatically be recorded for the autograder.

In [ ]:
# Do not change this cell; results are recorded for the autograder
Sources.dli.passes_step_2_with_n_of(1024)

### Check Your Work
Was the expected throughput achieved? Was the result still accurate? 
If not, try again until your code is fast enough and still correct. 
If your code passed, move on to the Step 3.


---

# Step 3: Coarse Grid with CUDA Kernel

At this point, our simulator is writing data to disk at the resolution of compute steps. 
We don't need such a fine resolution to visualize the results.
As part of this step, we'll try to coarse the grid.

<img src="Images/coarsening.png" alt="Coarsening" width=900>

We'll split the grid into 2D tiles and compute the average value of each tile.
For this step, please edit the cell below.
This cell already contains a CUDA kernel that assigns one thread block to one coarse tile.
Each thread has one value from the fine tile, which is already loaded for you:

```c++
float thread_value = fine(fine_row, fine_col);
```

Use `cub::BlockReduce` to compute the average value of all `thread_value`s in a given thread block and write the result into the coarse grid:
```c++
coarse(coarse_row, coarse_col) = block_average;
```

Note that `cub::BlockReduce` only returns a valid result in the first thread of the block.

<img src="Images/coop-reduce.png" alt="Reduce" width=600>

If you want to start again, just copy the backup source [Backup/coarse.cu](Backup/coarse.cu).

<details>
<summary>Original code in case you need it</summary>

```c++
%%writefile Sources/coarse.cu
#include "dli.h"

__global__ void kernel(dli::temperature_grid_f fine,
                       dli::temperature_grid_f coarse) {
  int coarse_row = blockIdx.x / coarse.extent(1);
  int coarse_col = blockIdx.x % coarse.extent(1);
  int row = threadIdx.x / dli::tile_size;
  int col = threadIdx.x % dli::tile_size;
  int fine_row = coarse_row * dli::tile_size + row;
  int fine_col = coarse_col * dli::tile_size + col;

  float thread_value = fine(fine_row, fine_col);

  // FIXME(Step 3):
  // Compute the sum of `thread_value` across threads of a thread block
  // using `cub::BlockReduce`
  float block_sum = ...;

  // FIXME(Step 3):
  // `cub::BlockReduce` returns block sum in thread 0, make sure to write
  // result only from the first thread of the block
  coarse(coarse_row, coarse_col) = block_average;
}

// Don't change the signature of this function
void coarse(dli::temperature_grid_f fine, dli::temperature_grid_f coarse) {
  kernel<<<coarse.size(), dli::block_threads>>>(fine, coarse);
}
```

</details>

In [ ]:
%%writefile Sources/coarse.cu
#include "dli.h"

__global__ void kernel(dli::temperature_grid_f fine,
                       dli::temperature_grid_f coarse) {
  int coarse_row = blockIdx.x / coarse.extent(1);
  int coarse_col = blockIdx.x % coarse.extent(1);
  int row = threadIdx.x / dli::tile_size;
  int col = threadIdx.x % dli::tile_size;
  int fine_row = coarse_row * dli::tile_size + row;
  int fine_col = coarse_col * dli::tile_size + col;

  float thread_value = fine(fine_row, fine_col);

  // FIXME(Step 3):
  // Compute the sum of `thread_value` across threads of a thread block
  // using `cub::BlockReduce`
  float block_sum = ...;

  // FIXME(Step 3):
  // `cub::BlockReduce` returns block sum in thread 0, make sure to write
  // result only from the first thread of the block
  coarse(coarse_row, coarse_col) = block_average;
}

// Don't change the signature of this function
void coarse(dli::temperature_grid_f fine, dli::temperature_grid_f coarse) {
  kernel<<<coarse.size(), dli::block_threads>>>(fine, coarse);
}

After you are done modifying the code, run the code cell below to see how your changes affect data that is stored to disk.

In [ ]:
Sources.dli.run_step_3()

Once the code is running error-free and you are seeing the visualization, run the code cell below to see how your changes affect performance and accuracy of the application.  
The result will automatically be recorded for the autograder.

In [ ]:
# Do not change this cell; results are recorded for the autograder
Sources.dli.passes_step_3_with_n_of(1024)

### Check Your Work
Was the expected throughput achieved? Was the result still accurate?  If not, try again until your code is fast enough and still correct.
If your code passed, move on to the submission step. 


---

# Step 4: Submit Your Assessment

If you are satisfied that you have completed the steps correctly, it is time to submit your project as follows to the autograder:

1. Go back to the DLI course GPU launch page and click the checkmark to run the assessment:

<img src="Images/assessment_checkmark.png">

2. That's it!  You'll receive a pop-up window with the grading feedback, and the points will be credited to your progress. 

<img src="Images/assessment_pass_popup.png">

You can always check your grade in the course progress tab.  Note that partial values for the coding assessment won't be visible here - it shows up as either 0 or 100 points. 

<img src="Images/progress.png">

 Once you've passed this assessment, you can find your certificate on the `My Learning` page, linked in the dropdown menu of your account.

 <img src="Images/dli_dropdown.png">

<a href="https://www.nvidia.com/dli"> <img src="Images/nvidia_header.png" alt="Header" style="width: 400px;"/> </a>